# Can You Snatch Defeat From The Jaws Of Victory?

**2020-06-12**: https://fivethirtyeight.com/features/can-you-snatch-defeat-from-the-jaws-of-victory/

## Riddler Express

Last Sunday we lost Alex Trebek, a giant in the world of game shows and trivia. The show he hosted over the course of four decades — Jeopardy! — has previously appeared in this column. Today, it makes a return.

You’re playing the (single) Jeopardy! round, and your opponents are simply no match for you. You choose first and never relinquish control, working your way horizontally across the board by first selecting all six \\$200 clues, then all six \\$400 clues, and so on, until you finally select all the \\$1,000 clues. You respond to each clue correctly before either of your opponents can.

One randomly selected clue is a Daily Double. Rather than award you the prize money associated with that clue, it instead allows you to double your current winnings or wager up to $1,000 should you have less than that. Being the aggressive player you are, you always bet the most you can. (In reality, the Daily Double is more likely to appear [in certain locations](https://www.vox.com/2015/3/3/8140405/jeopardy-daily-double-statistics) on the board than others, but for this problem assume it has an equal chance of appearing anywhere on the board.)

How much money do you expect to win during the Jeopardy! round?

Extra credit: Suppose you change your strategy. Instead of working your way horizontally across the board, you select random clues from anywhere on the board, one at a time. Now how much money do you expect to win during the Jeopardy! round?

### Solution

Let's just enumerate the outcomes.

In [1]:
import numpy as np

In [2]:
all_clues = np.repeat([200, 400, 600, 800, 1000], 6)
all_clues

array([ 200,  200,  200,  200,  200,  200,  400,  400,  400,  400,  400,
        400,  600,  600,  600,  600,  600,  600,  800,  800,  800,  800,
        800,  800, 1000, 1000, 1000, 1000, 1000, 1000])

Let's create an array of length 30 that represents how much money I'd have at each point in the game, assuming the daily double hasn't already been chosen. The index number represents how many questions I've answered so far. FOr example at index 0 it's the beginning of the game and I'm starting with \\$0. Index 6 has a value of \\$1200 because I've answered all six of the \\$200 questions.

In [3]:
round_start = np.cumsum(all_clues) - all_clues
round_start

array([    0,   200,   400,   600,   800,  1000,  1200,  1600,  2000,
        2400,  2800,  3200,  3600,  4200,  4800,  5400,  6000,  6600,
        7200,  8000,  8800,  9600, 10400, 11200, 12000, 13000, 14000,
       15000, 16000, 17000])

Now I'll create an array that represents how much I'd gain by the daily double appearing at that point in the game. It's simply the value of what I have at the time I select the daily double, unless I'm under \\$1000. If I don't yet have \\$1000 then the gain is \\$1000.

In [4]:
gain = np.where(round_start < 1000, 1000, round_start)
gain

array([ 1000,  1000,  1000,  1000,  1000,  1000,  1200,  1600,  2000,
        2400,  2800,  3200,  3600,  4200,  4800,  5400,  6000,  6600,
        7200,  8000,  8800,  9600, 10400, 11200, 12000, 13000, 14000,
       15000, 16000, 17000])

Now let's loop through every possible position for the Daily Double to appear in. The amount of money I'll have at the end of the round is the money I have when I select the Daily Double, plus the gain I receive by selecting it at that point, plus the sum of all clues left to select still.

In [5]:
totals = np.arange(30)

for i in range(30):
    start = round_start[i]
    profit = gain[i]
    total = start + profit + np.sum(all_clues[i+1:])
    totals[i] = total
    
totals

array([18800, 18800, 18800, 18800, 18800, 18800, 18800, 19200, 19600,
       20000, 20400, 20800, 21000, 21600, 22200, 22800, 23400, 24000,
       24400, 25200, 26000, 26800, 27600, 28400, 29000, 30000, 31000,
       32000, 33000, 34000])

Since the Daily Double is equally likely to appear in any position, the expected value is simply the average of all these final values. 

In [6]:
np.mean(totals)

23800.0

### Extra Credit Solution

Let's use a simulation.

In [44]:
n_sims = 3

clues = np.random.random(size=(n_sims, 30))
clues = all_clues[np.argsort(clues)]

round_start = np.cumsum(clues, axis=1) - clues
gain = np.where(round_start < 1000, 1000, round_start)
gain

array([[ 1000,  1000,  1000,  1600,  1800,  2000,  2600,  3200,  3600,
         4600,  5400,  6400,  6600,  6800,  7200,  8200,  9200,  9600,
        10400, 10600, 11000, 11400, 12400, 13000, 14000, 14400, 15200,
        15800, 16600, 17400],
       [ 1000,  1000,  2000,  2400,  3200,  4200,  4400,  4800,  5000,
         5600,  6000,  6400,  6600,  7400,  8200,  8800,  9000,  9200,
        10000, 10600, 11200, 11800, 12400, 13200, 14200, 15000, 15400,
        15800, 16000, 17000],
       [ 1000,  1000,  1000,  1400,  2400,  3200,  3600,  4600,  5400,
         6400,  6600,  7600,  8000,  8400,  9200, 10000, 10600, 11200,
        11600, 12600, 13000, 13800, 14000, 14600, 14800, 15600, 15800,
        16800, 17200, 17800]])

In [47]:
daily_double = np.random.randint(30, size=3)
daily_double

array([26, 20,  1])

In [51]:
profits = np.arange(n_sims)
profits

array([0, 1, 2])

In [ ]:
for i in range(n_sims):
    DD = daily_double[i]
    
    